### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Mesothelioma/GSE73161'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)
    

import numpy as np
nan = np.nan


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset likely contains gene expression data
if 'gene expression profiling' in "!Series_summary\t\"Characterization of ~ 68 cell lines derived from human sarcoma and 5 normal counterpart cells,  including drug sensitivity testing, gene expression profiling and microRNA expression profiling have been completed.".lower() or 'transcription profiles' in "!Series_summary\t\"Characterization of ~ 68 cell lines derived from human sarcoma and 5 normal counterpart cells,  including drug sensitivity testing, gene expression profiling and microRNA expression profiling have been completed.".lower():
    is_gene_available = True

# Check the availability of 'Mesothelioma', 'age', and 'gender' variables and their corresponding data type conversions
sample_characteristics = {
    0: ['cell line: MRC-9', 'cell line: NARna', 'cell line: NCI-H128', 'cell line: NCI-H146', 
        'cell line: NCI-H1688', 'cell line: NCI-H187', 'cell line: NCI-H1930', 'cell line: NCI-H196', 
        'cell line: NCI-H1963', 'cell line: NCI-H211', 'cell line: NCI-H2171', 'cell line: NCI-H28', 
        'cell line: NCI-H378', 'cell line: NCI-H446', 'cell line: NCI-H524', 'cell line: NCI-H526', 
        'cell line: NCI-H69', 'cell line: NCI-H82', 'cell line: NCI-H865', 'cell line: NCI-H889', 
        'cell line: NLRna1', 'cell line: NLRna2', 'cell line: WI-38', 'cell line: DMS 114', 
        'cell line: DMS 187', 'cell line: DMS 273', 'cell line: NCI-H209', 'cell line: NCI-H249', 
        'cell line: NCI-H250', 'cell line: NCI-H345'], 
    1: ['cell type: Fibroblast', 'cell type: Normal adrenal RNA', 'cell type: SCLC', 
        'cell type: Mesothelioma', 'cell type: Normal lung RNA', 'cell type: NSCLC', nan]
}

# 'Mesothelioma'
mesothelioma_values = [value.split(': ')[1] for value in sample_characteristics[1] if isinstance(value, str)]
if 'Mesothelioma' in mesothelioma_values:
    trait_row = 1

def convert_trait(value):
    try:
        val = value.split(': ')[1]
        return 1 if val == 'Mesothelioma' else 0
    except:
        return None

# 'age' not available based on given characteristics
age_row = None

def convert_age(value):
    return None

# 'gender' not available based on given characteristics
gender_row = None

def convert_gender(value):
    return None

# Save cohort information
save_cohort_info('GSE73161', './preprocessed/Mesothelioma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Mesothelioma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Mesothelioma/trait_data/GSE73161.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine which keys in the gene annotation data store identifiers and gene symbols
identifier_key = 'miRNA_ID'
gene_symbol_key = 'miRNA_ID'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# Print the columns of gene_mapping to verify
print("Gene mapping dataframe columns before renaming:")
print(gene_mapping.columns)

# Ensure the column names are correct in gene_mapping
gene_mapping = gene_mapping.rename(columns={identifier_key: 'ID'})

# Print the columns of gene_mapping to verify after renaming
print("Gene mapping dataframe columns after renaming:")
print(gene_mapping.columns)

# 3. Apply the mapping and update the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first few rows to verify the mapping
print("Gene data preview after mapping:")
print(preview_df(gene_data))


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Mesothelioma/gene_data/GSE73161.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Mesothelioma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE73161', './preprocessed/Mesothelioma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Mesothelioma/GSE73161.csv'
    unbiased_merged_data.to_csv(csv_path)
